In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [9]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.resize(img,(224,244), interpolation = cv2.INTER_AREA)
            img = img_to_array(img)
            img = preprocess_input(img)
            images.append(img)
    return images

X_train=load_images_from_folder('dataset_smoking_notsmoking_splitted/training_data/notsmoking/')+load_images_from_folder('dataset_smoking_notsmoking_splitted/training_data/smoking/')
X_valid=load_images_from_folder('dataset_smoking_notsmoking_splitted/validation_data/notsmoking/')+load_images_from_folder('dataset_smoking_notsmoking_splitted/validation_data/smoking/')
X_test=load_images_from_folder('dataset_smoking_notsmoking_splitted/testing_data/notsmoking/')+load_images_from_folder('dataset_smoking_notsmoking_splitted/testing_data/smoking/')

In [10]:
len(X_train),len(X_test),len(X_valid)

(1608, 396, 399)